# Kode for datafortelling om operative mål i PO Pensjon
Målingene legges manuelt inn i et excel dokument som må lastes opp som "../data/Måleparametere PO Pensjon 2022.csv" ved oppdatering.

In [ ]:
import sys
import importlib
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
from typing import Dict

import datastory as ds

sys.path.append('../lib')
import operative as opera
importlib.reload(opera)

In [ ]:
df = pd.read_csv("../data/Måleparametere PO Pensjon 2022.csv", sep=";", header=2)

In [ ]:
df = df.drop([0] + list(range(4,12)) + list(range(17,20)) + list(range(22,26))).reset_index(drop=True)

In [ ]:
df.head(30)

In [ ]:
df["prosent_flagg"] = df.Nullpunktsmåling.apply(lambda x: "%" in x)

In [ ]:
# Propager operative mål:
tmp = df["Operativt mål"].isna()
for i in range(df.shape[0]):
    if tmp[i]:
        df.iloc[i, 1] = df.iloc[i-1, 1]

In [ ]:
def clean_numbers(x):
    if isinstance(x, float):
        return x
    if "%" in str(x):
        tmp = x[:-2]
        x = str(float(tmp)/100)
    if "-" in str(x):
        x = "0"
    return "".join(x.split(" "))

In [ ]:
def set_plus(x):
    if not pd.isna(x):
        if x[0].isnumeric():
            return "+" + x
        elif x.strip() == "-":
            return "+0 %"
        else:
            return x

In [ ]:
# Få alle tall på riktig format
maalinger_cols = ['Nullpunktsmåling', 'Måling T1', 'Måling T2'] # Legg til nye målinger her
for col in maalinger_cols:
    df[col] = df[col].apply(lambda x: clean_numbers(x)).astype("float")

prosent_endring_cols = ['% endring mot NP', '% endring mot NP.1'] # Legg til ny endringsprosent hver tertial
for col in prosent_endring_cols:
    df[col] = df[col].apply(lambda x: set_plus(x))

In [ ]:
df.tail()

In [ ]:
# Hardkoder en linje fordi NP er "-" og ikke "0%":
df.loc[df.shape[0]-1, "prosent_flagg"] = True     

### Fjern målepunkter med bare "-" (hvis ønskelig):
#df.loc[df.shape[0]-1, "Nullpunktsmåling"] = np.nan
#df.drop([0,7,11,17,18,19,28] , inplace=True)

In [ ]:
def break_lines(string, max_letters):
    N = len(string)
    if len(string) > max_letters:
        i = max_letters
        while i < N and string[i] != " ":
            i += 1
        if i < N:
            string = string[:i] + '<br>' + string[i:]
    return string

In [ ]:
df["Måleparameter"] = df.Måleparameter.apply(lambda x: break_lines(x, 50))

In [ ]:
mp_per_op = df.groupby("Operativt mål")["Måleparameter"].count().to_dict()

In [ ]:
df = df.T

In [ ]:
df

In [ ]:
operatives = []
prev_op = ""
j = 1
for i in range(df.shape[1]):
    col = df.iloc[:,i]
    op = col["Operativt mål"]
    mparam = col["Måleparameter"]
    new_op = op != prev_op
    if new_op:
        print(op)
        j = 1
        # Append objekt for opprettet mål
        if prev_op != "":
            operatives.append(Op)
        # Lag objekt for nytt operativt mål
        Op = opera.Operative(op, mp_per_op[op])
    prev_op = op
    Op.MPs.append(opera.MP.from_series(col=col, i=j, n=len(operatives)%7))
    j += 1
# Append siste operative mål
operatives.append(Op)

In [ ]:
story = ds.DataStory("Operative mål i PO Pensjon")
story.markdown("# Operative mål i PO Pensjon")
story.markdown("Oversikt over operative mål i PO Pensjon og tilhørende målepunkter. Enkelte målepunkter er ikke inkludert her. Prosent i plottene angir endring fra nullpunktsmåling.")
for op in operatives:
    story.header(op.name, level=1)
    print(op.name)
    op.big_fig = op.make_big_fig()
    story.plotly(op.big_fig.to_json())
    
    mp_text = ""
    
    for mp in op.MPs:
        if not pd.isna(mp.col.Beskrivelse):
            mp_text += f"**{mp.name.replace('<br>', '')}:** " + mp.col.Beskrivelse  
        if not pd.isna(mp.col.Kommentar):
            mp_text += "  \n *Kommentar:* " + mp.col.Kommentar
        mp_text += "  \n  \n"
        print(mp.name)
    story.markdown(mp_text)

In [ ]:
#story.publish(url="https://nada.ekstern.dev.nav.no/api") #dev
story.publish(url="https://nada.intern.nav.no/api") #prod
#story.update(url="https://nada.intern.nav.no/api", token='ed32f304-8fc4-489f-b08d-c6b1b719f22e')

# Make dataproduct

In [ ]:
df.T.head()

In [ ]:
df_dp = (
    df
    .T
    .drop(["NP gjennomført", "% endring mot NP", "% endring mot NP.1", "% endring mot NP.2", "Måling T3"], axis=1)
    .rename(columns = {"Måleparameter": "MP",
           "Operativt mål": "OM",
           "Nullpunktsmåling": "NP",
           "Måling T1": "T1",
           "Måling T2": "T2"
                      })
    .melt(id_vars=["MP", "OM", "Beskrivelse", "Kommentar", "prosent_flagg"],
          var_name="Tertial",
          value_name='Verdi')
    .sort_values(["OM", "MP"]).
    reset_index(drop=True)
)

In [ ]:
df_dp.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'teampensjon-prod-cebd.operative_mal.mal22'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="teampensjon-prod-cebd")

job = client.load_table_from_dataframe(df_dp, table_id, job_config=job_config)
job.result()